# Need to figure out how to confirm that the format is correct...

In [ ]:
# FORMAT TESTING....

In [1]:
def dup_entries(df):
    duplicates = df[df.duplicated(subset=['racer_id', 'discipline'], keep=False)]
    
    if not duplicates.empty:
        print("Found duplicates for (racer_id, discipline):")
        return duplicates
    else:
        print("No duplicates found.")
        return None

# Create the connection

In [3]:
import pandas as pd
import sqlite3
from helper_functions import upload_new_race_results

# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# Upload timetrial data (just race results)

In [4]:
import pandas as pd
import numpy as np
from helper_functions import clean_string, RACE_RESULT_COL_ORDER, upload_results

In [5]:
result_cols = ["bib", "discipline", "racer_id", "run1", "run2"]
df_load = pd.read_csv("2025/241229FnGRaceSimResultsBoth.csv", header=None, names=result_cols)

In [6]:
df_load["racer_id"] = df_load["racer_id"].apply(clean_string)
dup_entries(df_load)
df_load.replace({'DNF': 9998,"DSQ": 9998, "DNS": 9999, pd.NA: 9999}, inplace=True)

No duplicates found.


In [7]:
df_load.head(2)

,bib,discipline,racer_id,run1,run2
0,1,SKI,alexhenry,9999,9999
1,2,SKI,tonypetrella,9999,9999


In [8]:
race_date = '12/29/2024'
race_id = 241229
description = '2025 Race Sim (time trials)'

df_load['run1'] = pd.to_numeric(df_load['run1']) # errors='coerce')
df_load['run2'] = pd.to_numeric(df_load['run2']) # errors='coerce')
df_load['best_time'] = df_load[['run1', 'run2']].min(axis=1)
df_load["points"] = np.nan
df_load["team"] = np.nan
df_load["tier"] = np.nan
df_load["race_id"] = race_id

In [9]:
df_load[RACE_RESULT_COL_ORDER].head(2)

,racer_id,discipline,team,tier,run1,run2,best_time,points,race_id,bib
0,alexhenry,SKI,NaN,NaN,9999.0,9999.0,9999.0,NaN,241229,1
1,tonypetrella,SKI,NaN,NaN,9999.0,9999.0,9999.0,NaN,241229,2


In [12]:
#upload_results(
#    df_load[RACE_RESULT_COL_ORDER],
#    race_id,
#    race_date,
#    description,
#    conn
#)

# Test the upload:

In [15]:
from queries import (
    get_point_total,
    audit_df,
    get_races_list,
    get_race_data,
    get_race_years
)

In [20]:
df = pd.read_sql("""
    SELECT substr(description, 1,4) as year
    FROM Races
    GROUP BY 1
    ORDER BY CAST(year AS INT) DESC
    """,
    conn
)
df['year'].tolist()

['2025',
 '2024',
 '2023',
 '2022',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013']

In [18]:
years = get_race_years(conn)

In [19]:
years

['2024',
 '2023',
 '2022',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013']

In [16]:
race_list = get_races_list(conn)

In [17]:
race_list

,race_id,race_date,description,year
0,241229,2024-12-29 00:00:00,2025 Race Sim (time trials),2025
1,240218,2024-02-18 00:00:00,2024 F&G Race #4,2024
2,240211,2024-02-11 00:00:00,2024 F&G Race #3,2024
3,240121,2024-01-21 00:00:00,2024 F&G Race #2,2024
4,240114,2024-01-14 00:00:00,2024 F&G Race #1,2024
5,3039,2023-02-19 00:00:00,2023 F&G Race#4,2023
6,3038,2023-02-12 00:00:00,2023 F&G Race#3,2023
7,3037,2023-02-05 00:00:00,2023 F&G Race#2,2023
8,3036,2023-01-15 00:00:00,2023 F&G Race#1,2023
9,3035,2022-12-31 00:00:00,2023 Time Trials,2023


# Update team data (new year start list!)

# Upload race data (calc points)

In [ ]:
path = 'data/2024/240218FnGResultsBoth.csv'
race_date = '02/18/2024'
race_id = 240218
description = '2024 F&G Race #4'
N_tiers=13
N_teams=8 

""" # Comment out for now
upload_new_race_results(
    path,
    race_date,
    race_id,
    description,
    N_tiers,
    N_teams,
    conn
)
"""